## Key words
### 분산 분석, 회귀 분석

### 1. 고객의 총 송금액이 교육 수준, 혼인 여부에 따라서 어떤 특징을 보이는지 분산분석을 통해 알아보고자 한다. 
### 1회 평균 송금액을 종속변수로 했을 때 독립변수간 교호작용 여부를 알아보고 해당 p-value를 반올림하여 소수점 둘 째 자리 까지 기술하시오. (정답예시 : 0.12)

In [43]:
import pandas as pd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [44]:
df = pd.read_csv("financial_info_10k_persons.csv")
df.head(2)

,ID,is_attrited,Age,Gender,Dependent_cnt,Edu_level,Marital_status,Income,Card,Period_m,Total_rel_cnt,Inactive_last_12m,Contacts_cnt_last_12m,Credit_limit,Total_trans_amt,Total_trans_cnt
0,1,0,41,F,2,High School,Married,Less than $40K,Blue,36,6,2,2,4953.0,4183,67
1,2,0,38,M,0,High School,Single,$80K - $120K,Blue,29,3,3,2,5983.0,4141,65


In [45]:
df_q1 = df.copy()

In [46]:
model = ols(data = df, formula = "Total_trans_amt ~ Edu_level + Marital_status + Edu_level:Marital_status").fit() 
# 객체:객체는 교호작용
anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
Edu_level,6.0,7.145069e+07,1.190845e+07,1.139094,3.365109e-01
Marital_status,3.0,4.442492e+08,1.480831e+08,14.164784,3.289793e-09
Edu_level:Marital_status,18.0,1.853456e+08,1.029698e+07,0.984950,4.737308e-01
Residual,9972.0,1.042504e+11,1.045431e+07,NaN,NaN


In [47]:
anova_lm(model).round(2) # 정답 : 0.47

,df,sum_sq,mean_sq,F,PR(>F)
Edu_level,6.0,7.145069e+07,1.190845e+07,1.14,0.34
Marital_status,3.0,4.442492e+08,1.480831e+08,14.16,0.00
Edu_level:Marital_status,18.0,1.853456e+08,1.029698e+07,0.98,0.47
Residual,9972.0,1.042504e+11,1.045431e+07,NaN,NaN


### 2. 고객의 신용 한도는 다양한 정보를 기반으로 결정된다.
### 고객의 금융활동이 누적됨에 따라 신용 한도는 바뀌기도 하는데 이를 비교하고자 한다.
### 고객의 신용한도를 종속변수를 공통으로 하고 부양가족, 나이, 학력, 성별, 결혼여부를 1번 회귀모델.
### 1번 모델에 가입기간과 누적 송금 횟수를 독립변수에 더한 회귀 모델의 결정계수 차이의 절대값을 반올림하여 소수점 셋 째 자리까지 기술하시오. (정답 예시 : 0.123)

In [75]:
df_q2 = df.copy()
df_q2.head(2)

,ID,is_attrited,Age,Gender,Dependent_cnt,Edu_level,Marital_status,Income,Card,Period_m,Total_rel_cnt,Inactive_last_12m,Contacts_cnt_last_12m,Credit_limit,Total_trans_amt,Total_trans_cnt
0,1,0,41,F,2,High School,Married,Less than $40K,Blue,36,6,2,2,4953.0,4183,67
1,2,0,38,M,0,High School,Single,$80K - $120K,Blue,29,3,3,2,5983.0,4141,65


In [76]:
y = df_q2["Credit_limit"]
X1 = pd.concat([y, df_q2.loc[:, "Age":"Marital_status"]], axis = 1)
X2 = pd.concat([X1, df_q2.loc[:, ["Period_m", "Total_trans_cnt"]]], axis = 1)
print(X1.columns)
print(X2.columns)

Index(['Credit_limit', 'Age', 'Gender', 'Dependent_cnt', 'Edu_level',
       'Marital_status'],
      dtype='object')
Index(['Credit_limit', 'Age', 'Gender', 'Dependent_cnt', 'Edu_level',
       'Marital_status', 'Period_m', 'Total_trans_cnt'],
      dtype='object')


In [77]:
X1_dum = pd.get_dummies(X1, drop_first= True)
X1_dum.head(2)

,Credit_limit,Age,Dependent_cnt,Gender_M,Edu_level_Doctorate,Edu_level_Graduate,Edu_level_High School,Edu_level_Post-Graduate,Edu_level_Uneducated,Edu_level_Unknown,Marital_status_Married,Marital_status_Single,Marital_status_Unknown
0,4953.0,41,2,0,0,0,1,0,0,0,1,0,0
1,5983.0,38,0,1,0,0,1,0,0,0,0,1,0


In [81]:
X1_dum = X1_dum.rename(columns = {"Edu_level_High School": "Edu_level_High_School",
                                  "Edu_level_Post-Graduate": "Edu_level_Post_Graduate"})

In [82]:
"Credit_limit ~ " + " + ".join(X1_dum.columns[1:])

'Credit_limit ~ Age + Dependent_cnt + Gender_M + Edu_level_Doctorate + Edu_level_Graduate + Edu_level_High_School + Edu_level_Post_Graduate + Edu_level_Uneducated + Edu_level_Unknown + Marital_status_Married + Marital_status_Single + Marital_status_Unknown'

In [83]:
model_1 = ols(formula = "Credit_limit ~ " + " + ".join(X1_dum.columns[1:]), data = X1_dum).fit()

In [84]:
X2_dum = pd.get_dummies(X2, drop_first= True)
X2_dum.head(2)

,Credit_limit,Age,Dependent_cnt,Period_m,Total_trans_cnt,Gender_M,Edu_level_Doctorate,Edu_level_Graduate,Edu_level_High School,Edu_level_Post-Graduate,Edu_level_Uneducated,Edu_level_Unknown,Marital_status_Married,Marital_status_Single,Marital_status_Unknown
0,4953.0,41,2,36,67,0,0,0,1,0,0,0,1,0,0
1,5983.0,38,0,29,65,1,0,0,1,0,0,0,0,1,0


In [85]:
X2_dum = X2_dum.rename(columns = {"Edu_level_High School": "Edu_level_High_School",
                                  "Edu_level_Post-Graduate": "Edu_level_Post_Graduate"})

In [87]:
model_2 = ols(formula = "Credit_limit ~ " + " + ".join(X2_dum.columns[1:]), data = X2_dum).fit()

In [88]:
model_1.rsquared

0.18542639981257603

In [89]:
model_2.rsquared

0.19413399639018658

In [91]:
round(abs(model_1.rsquared - model_2.rsquared), 3) # 정답 : 0.009

0.009

### 3. 신규 고객이 개인정보를 입력할 경우 예상 신용 한도를 보여주려고한다.
### 부양가족과 수입이 없는 29세 고졸(High School) 미혼의 남성의 경우 예상 신용 한도를 정수부분만 기술

In [92]:
X1_dum.head(1)

,Credit_limit,Age,Dependent_cnt,Gender_M,Edu_level_Doctorate,Edu_level_Graduate,Edu_level_High_School,Edu_level_Post_Graduate,Edu_level_Uneducated,Edu_level_Unknown,Marital_status_Married,Marital_status_Single,Marital_status_Unknown
0,4953.0,41,2,0,0,0,1,0,0,0,1,0,0


In [96]:
X1_test = X1_dum.iloc[[0], 1:] # 데이터프레임 유지하면서 하나만들고오기 [0]
X1_test["Age"] = 29
X1_test["Dependent_cnt"] = 0
X1_test["Gender_M"] = 1
X1_test["Marital_status_Married"] = 0
X1_test["Marital_status_Single"] = 1

In [95]:
X1_test

,Age,Dependent_cnt,Gender_M,Edu_level_Doctorate,Edu_level_Graduate,Edu_level_High_School,Edu_level_Post_Graduate,Edu_level_Uneducated,Edu_level_Unknown,Marital_status_Married,Marital_status_Single,Marital_status_Unknown
0,29,0,1,0,0,1,0,0,0,0,1,0


In [98]:
model_1.predict(X1_test) # 정답 : 11403

0    11403.358901
dtype: float64